In [1]:
#//*** This thread helped with the connection
#//***https://stackoverflow.com/questions/37692780/error-28000-login-failed-for-user-domain-user-with-pyodbc

#//**** Update the ODBC Driver
#//**** https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15
import pyodbc
import datetime
from datetime import date, timedelta
print(pyodbc.drivers())
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:OM-CASF-DB01' 
server = 'OM-CASF-DLSQL' 
# server = '10.218.97.2'
database = 'DaletDB' 
#username = 'SWNA\stonk013' 
#password = 'inara51More' 
username = '$kgotvscripts' 
password = 'ABC7news' 

#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cnxn = pyodbc.connect('Trusted_Connection=yes;DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client 11.0']


### SQL Query for MOS Active rundowns ###

In [2]:
startDate = date.today() + timedelta(days=-30)
endDate = date.today() + timedelta(days=30)
query = f"""
select DISTINCT Clocks.clock_date AS Rundown_date_time, titles.title Rundown_Name, MosDevices.Name, STAT.Content Status, Content
from playoutItemsStatus STAT, clocks, MosDevices, titles 
where
    DeviceType='GatewayItemInfo'
and STAT.ClockId = clocks.clock_id 
and clocks.clock_id = titles.title_id
and Clocks.clock_date > '{startDate}'
and Clocks.clock_date <= '{endDate}'
order by Rundown_date_time;
"""
cursor.execute(query)
tables = cursor.fetchall()
tdict={ "ALEX TEST RUNDOWN" : ""}
for row in tables:
    if '<GatewayItemInfo status.l="10">' in row[3]:
        if row[1] not in tdict.keys():
            tdict[row[1]] = ""
            print(datetime.datetime.date(row[0]),row[1])

2021-03-28 5PM Weekend
2021-03-31 11AM Midday
2021-03-31 11:30AM Midday
2021-04-03 Dalet OD XPression
2021-04-07 Promo
2021-04-08 11PM WEEKDAY


In [104]:

query = """
select TOP 10 *
from playoutItemsStatus 

"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row)



    
query = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'playoutItemsStatus '
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row[2],row[3])

query = """
select *
from playoutItemsStatus 
WHERE DeviceType LIKE 'GatewayItemInfo' and DeviceId = '25'
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row)

(29610532, '29610532:1000055487', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29610532, '29610532:1000055487', 'ShowTimeTrackingProgramInfo', 'ShowTimeTracking', '<ShowTimeTrackingProgramInfo actualAiredStartTime.td="2021-02-25T17:59:04.996" onairStatus.l="0" startTrackWithBeginTime.l="0" trackUserId.l="-1" triggerStatus.l="1"/>')
(29610532, '29610532:1000055487:815351', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedStartTime.td="2021-02-25T17:59:04.996" automationLock.l="0" currentStory2Offset.tc="P0.000S" isCurrTrackItem.l="0"/>')
(29610532, '29610532:1000055487:815351:S0:1', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29610532, '29610532:1000055487:815351:S1:2', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription

In [49]:
query = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'MosDevices'
"""
cursor.execute(query)
tables = cursor.fetchall()
header = ""
print("======================")
print(tables[0][2])
print("======================")
for row in tables:
    header += row[3] + ", "
    
print(header[:-2])
query = """
select *
from MosDevices 
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row[0],row[1],row[2],row[3],row[4],row[5],row[6])


MosDevices
Id, Type, Name, Description, HostId, IsIncremental, PreloadByBroadcastServer, Icon, Params
19 22 kgo.pcr2.ovd.mos Ross OD PCR 2 207 1 0
20 22 kgo.pcr3.ovd.mos Ross OD PCR 3 213 1 0
23 22 kgo.pcr2.ovdbu.mos Ross Overdrive PCR2 Backup 205 1 0
25 25 webmos.appdev.kgo.abc.mos WEBMOS Prompter 210 1 0
22 42 xpn.kgo.abc.mos Ross Xpression 201 1 0
14 25 bdl.promptera.kgo.abc.mos Prompter 1 211 1 0
24 22 kgo.pcr3.ovdbu.mos Ross Overdrive PCR3 Backup 206 1 0
15 25 bdl.prompterb.kgo.abc.mos Prompter 2 212 1 0


In [107]:
query = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'playoutItemsStatus STAT'
"""
cursor.execute(query)
tables = cursor.fetchall()
header = ""
print("======================")
print(tables)
print("======================")
for row in tables:
    header += row[3] + ", "
    
print(header[:-2])
query = """
select *
from playoutItemsStatus STAT 
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row)


[]

(29610532, '29610532:1000055487', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29610532, '29610532:1000055487', 'ShowTimeTrackingProgramInfo', 'ShowTimeTracking', '<ShowTimeTrackingProgramInfo actualAiredStartTime.td="2021-02-25T17:59:04.996" onairStatus.l="0" startTrackWithBeginTime.l="0" trackUserId.l="-1" triggerStatus.l="1"/>')
(29610532, '29610532:1000055487:815351', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedStartTime.td="2021-02-25T17:59:04.996" automationLock.l="0" currentStory2Offset.tc="P0.000S" isCurrTrackItem.l="0"/>')
(29610532, '29610532:1000055487:815351:S0:1', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29610532, '29610532:1000055487:815351:S1:2', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescrip

(29807882, '29807882:1000055856:824361', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedDuration.null="" airedStartTime.null="" automationLock.null="" currentStory2Offset.tc="P0.000S" isCurrTrackItem.null=""/>')
(29807882, '29807882:1000055856:824362', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedDuration.null="" airedStartTime.null="" automationLock.null="" currentStory2Offset.tc="P0.000S" isCurrTrackItem.null=""/>')
(29807882, '29807882:1000055857:824365', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedDuration.null="" airedStartTime.null="" automationLock.null="" currentStory2Offset.tc="P0.000S" isCurrTrackItem.null=""/>')
(29807882, '29807882:1000055857:824366', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedDuration.null="" airedStartTime.null="" automationLock.null="" currentStory2Offset.tc="P0.000S" isCurrTrackItem.null=""/>')
(29807882, '29807882

(29823600, '29823600:1000055890:827345:S3:4', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29823600, '29823600:1000055890:827345:S4:5', 'GatewayItemInfo', '23', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29823600, '29823600:1000055890:827345:S4:5', 'GatewayItemInfo', '24', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusDescription></GatewayItemInfo>')
(29823600, '29823600:1000055890:827346', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedStartTime.td="2021-03-15T16:31:43.837" automationLock.l="0" currentStory2Offset.tc="P0.000S" isCurrTrackItem.l="0"/>')
(29823600, '29823600:1000055890:827346:S0:1', 'GatewayItemInfo', '23', '<GatewayItemInfo status.l="3"><statusDescription t="ws">got no reply for rundown sent</statusD

(29840848, '29840848:1000056285:838879:S5:9', 'GatewayItemInfo', '19', '<GatewayItemInfo status.l="10"><statusDescription t="ws">Acknowledged by device</statusDescription></GatewayItemInfo>')
(29840848, '29840848:1000056285:838879:S5:9', 'GatewayItemInfo', '20', '<GatewayItemInfo status.l="10"><statusDescription t="ws">Acknowledged by device</statusDescription></GatewayItemInfo>')
(29840848, '29840848:1000056285:838884', 'GatewayItemInfo', '15', '<GatewayItemInfo status.l="10"><statusDescription t="ws">Acknowledged by device</statusDescription></GatewayItemInfo>')
(29840848, '29840848:1000056285:838884', 'GatewayItemInfo', '25', '<GatewayItemInfo status.l="10"><statusDescription t="ws">Acknowledged by device</statusDescription></GatewayItemInfo>')
(29840848, '29840848:1000056285:838884:S0:2', 'GatewayItemInfo', '19', '<GatewayItemInfo status.l="10"><statusDescription t="ws">Acknowledged by device</statusDescription></GatewayItemInfo>')
(29840848, '29840848:1000056285:838884:S0:2', 'Gat

In [ ]:
query = """
SELECT
  *
FROM
  INFORMATION_SCHEMA.TABLES
"""

cursor.execute(query)
tables = cursor.fetchall()

In [ ]:
master_tables = []
for table in tables:
    master_tables.append(table[2])
master_tables.sort()

In [ ]:
for table in master_tables:
    print(table)

In [ ]:
query = """
SELECT
*
FROM
RundownViews
"""
cursor.execute(query)
results = cursor.fetchall()
for x in results:
    print(x)

In [ ]:
query = """
SELECT
*
FROM
StoryContent
WHERE
TitleId='29807928'
"""
cursor.execute(query)
results = cursor.fetchall()
for x in results:
    for y in x:
        print(y)
    

In [ ]:
 
query = """
SELECT *
FROM RundownViewsAssociations 
"""
cursor.execute(query)
results = cursor.fetchall()


In [ ]:
print ([x[0] for x in cursor.description])
for row in results:
    print(row)

In [11]:
query = """
SELECT * 
FROM INFORMATION_SCHEMA.COLUMNS 
"""
cursor.execute(query)
results = cursor.fetchall()

#for row in results:
#    print(row[3])

In [12]:
titleid_dict = {}
#//*** Script ID 29807928
#//*** Rundown ID 29620400
for row in results:
    if  row[7] == 'int':
        #print(row[2],row[3])
        loop_query = f"""
        SELECT count(*)
        FROM {row[2]}
        WHERE {row[3]}='29620400' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()[0][0]
        if loop_result > 0:
            print(row[2],row[3],loop_result)
            titleid_dict[row[2]] = row[3]

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 33
TitleDataDictString TitleId 29
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 1
items clock_id 7
PurgeLog objectId 1
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
titles_references title_id 1
RecycleBin Object_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
PlayoutItemsStatus ClockId 57
clocks clock_id 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1


In [ ]:
query = """
SELECT * 
FROM INFORMATION_SCHEMA.COLUMNS 
"""
cursor.execute(query)
results = cursor.fetchall()
titleid_dict = {}
#//*** Script ID 29807928
#//*** Rundown ID 29620400
for row in results:
    #print(row)
    if  row[7] == 'int':
        #print(row[2],row[3])
        loop_query = f"""
        SELECT count(*)
        FROM {row[2]}
        WHERE {row[3]}='29807930' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()[0][0]
        if loop_result > 0:
            print(row[2],row[3],loop_result)
            titleid_dict[row[2]] = row[3]

In [ ]:
for table,column in titleid_dict.items():
        loop_query = f"""
        SELECT *
        FROM {table}
        WHERE {column}='29807930' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()
        print(f"{table} -- {column}")
        print ([x for x in cursor.description])
        
        for row in loop_result:
            print(row)
        print("=============")
    

In [16]:
def hunt_db(input_search,input_type='int'):
    query = """
    SELECT * 
    FROM INFORMATION_SCHEMA.COLUMNS 
    """
    cursor.execute(query)
    results = cursor.fetchall()
    titleid_dict = {}
    #//*** Script ID 29807928
    #//*** Rundown ID 29620400
    for row in results:
        #print(row[2], row[3])
        if  row[7] == input_type:
            #print(row[2],row[3])
            loop_query = f"""
            SELECT count(*)
            FROM {row[2]}
            WHERE {row[3]}='{input_search}' 
            """
            cursor.execute(loop_query)
            loop_result = cursor.fetchall()[0][0]
            if loop_result > 0:
                print(row[2],row[3],loop_result)
                titleid_dict[row[2]] = row[3]
                
    for table,column in titleid_dict.items():
        loop_query = f"""
        SELECT *
        FROM {table}
        WHERE {column}='{input_search}' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()
        print(f"{table} -- {column}")
        print ([x for x in cursor.description])

        for row in loop_result:
            print(row)
        print("=============")

            
            
            
#hunt_db('29807933')

#//*** Hunt for BRIO
#hunt_db('29545833')

hunt_db('30213979')
#hunt_db('1173')

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 42
TitleDataDictInteger NumValue 4
TitleDataDictString TitleId 32
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 1
Versions OrginalTitleId 4
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 2
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 3
Titles_ExFields -- Title_id
[('Title_id', <class 'int'>, None, 10, 10, 0, False), ('Language_id', <class 'int'>, None, 10, 10, 0, False), ('Topic', <class 'str'>, None, 32, 32, 0, True), ('VersionComment', <class 'str'>, None, 1073741823, 1073741823, 0, True)]
(30213979, 99, None, None)
TitleDataDictInteger -- NumValue
[('DataField_Id', <class 'int'>, None, 10, 10, 0, False), ('TitleId', <class 'int'>, None, 10, 10, 0, False), ('ValueOrder', <class 'int'>, None, 10

In [ ]:
#select  count(*)
#from playoutItemsStatus, clocks, MosDevices, titles 
#where content like '%onairstatus%' and STAT.ClockId = clocks.clock_id and clocks.clock_id = titles.title_id
#order by Rundown_date_time desc ;
query = """
SELECT *
FROM playoutItemsStatus
"""
cursor.execute(query)
results = cursor.fetchall()
for x in results:
    print(x)

    

In [14]:
# Datafields are all the Locking metadata fields. This gets a list of all of them

query = f"""
SELECT *
FROM DataFields 
"""
cursor.execute(query)
results = cursor.fetchall()
for x in results:
    print(x)
#titles -- title_id

(1, 'Title Id', 'TitleId', 'Number', None, 1, 1, 1, 1, 1, None, None, None, 1, None, 'Number', '<FieldObjectTypesAllTypes/>', None, None, 0, None, None, None, None, None)
(2, 'Name', 'TitleName', 'ShortText', None, 1, 0, 1, 1, None, None, None, None, 1, None, 'String', '<FieldObjectTypesAllTypes/>', None, None, 0, None, None, None, None, None)
(3, 'Title Author', 'TitleAuthor', 'ShortText', None, 1, 0, 1, None, None, None, None, None, 1, None, 'String', '<FieldObjectTypesAllTypes/>', None, None, 0, None, None, None, None, None)
(4, 'Title Duration', 'TitleDuration', 'Duration', None, 1, 0, 1, None, 1, None, None, None, 1, None, 'Period', '<FieldObjectTypesAllTypes/>', None, None, 0, None, None, None, None, None)
(5, 'Title Frame Rate', 'TitleFrameRate', 'Number', None, 1, 0, 1, None, 1, None, None, None, 1, None, 'Number', '<FieldObjectTypesAllTypes/>', None, None, 0, None, None, None, None, None)
(6, 'Title Frame Rate Scale', 'TitleFrameRateScale', 'Number', None, 1, 0, 1, None, 1, No